En esta lección veremos como utilizar la función `AI_COMPLETE` pasándole una imagen y un prompt y además un prompt tipo objeto que admita varias imágenes y texto. 

## Imagen y un prompt

### Sintaxis

La función contiene dos argumentos obligatorios y cuatro opcionales. La sintaxis para el uso de `AI_COMPLETE` con una sola imagen como entrada es la siguiente:

```sql
AI_COMPLETE(
    <model>, <prompt>, <file> [, <model_parameters> ] )
```

### Modelos disponibles

- `claude-4-opus`
- `claude-4-sonnet`
- `claude-3-7-sonnet`
- `claude-3-5-sonnet`
- `llama-4-maverick`
- `llama-4-scout`
- `pixtral-large`

### Notas de uso

- Solo se admiten texto e imágenes. No se admiten archivos de vídeo ni audio.
- Se admiten imágenes con extensiones de archivo .jpg, .jpeg, .png, .gif y .webp. `pixtral-large` y `llama4` también admite .bmp.
- El tamaño máximo de la imagen es de 10 MB para la mayoría de los modelos y de 3,75 MB para los modelos Claude. Los modelos Claude no admiten imágenes con resoluciones superiores a 8000x8000.
- El stage que contiene las imágenes debe tener habilitado el cifrado del lado del servidor. No se admiten stages cifrados del lado del cliente.
- La función no admite políticas de red personalizadas.
- En los nombres de los stages no hay diferencias entre mayúsculas y minúsculas, pero en las rutas sí.

### Ejemplos

Para estos ejemplos vamos a crear un stage que cumpla con las características necesarias y luego cargaremos a ese stage las imagenes que usaremos para los ejemplos.

In [ ]:
-- Crear un internal stage con directory table activado y encriptación del lado del servidor

CREATE OR REPLACE STAGE curso_ia.seccion_2.imagenes
 DIRECTORY = (ENABLE = TRUE)
 ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');

**En este paso usted debe descargar las imágenes adjuntas como recurso a esta lección y cargarlas al stage que acabamos de crear.**

In [ ]:
USE DATABASE curso_ia;
USE SCHEMA seccion_2;

SELECT AI_COMPLETE('claude-3-5-sonnet',
    'Durante los primeros 200 días de mandato, ¿qué presidente logró que el SP500 tuviera un mejor desempeño?',
    TO_FILE('@IMAGENES', 'inaguration_day.png'));

**Clasificación de imágenes**

Este ejemplo clasifica el punto de referencia identificado en una sola imagen.

In [ ]:
USE DATABASE curso_ia;
USE SCHEMA seccion_2;

SELECT AI_COMPLETE('claude-3-5-sonnet',
    'Clasifique el punto de referencia identificado en esta imagen. Responda en JSON solo con el nombre del punto de referencia.',
    TO_FILE('@imagenes', 'soumaya.jpg'));

**Extracción de entidades de una imagen**

Este ejemplo extrae las entidades (objetos) de una imagen y devuelve los resultados en formato JSON.

In [ ]:
USE DATABASE curso_ia;
USE SCHEMA seccion_2;

SELECT AI_COMPLETE('claude-3-5-sonnet',
    'Extraiga los autos identificados en esta imagen. Responda en JSON solo con los autos identificados.',
    TO_FILE('@imagenes', 'autos.jpg'));

## Prompt tipo objeto

Esta alternativa de la función `AI_COMPLETE` genera una respuesta para un un prompt tipo objeto. El prompt tipo objeto hace referencia a una o más columnas que contienen datos de texto o imagen.

### Sintaxis

```sql
AI_COMPLETE(
    <model>, <prompt> [ , <model_parameters> ] )
```

Para conocer la lista de modelos disponibles que se pueden utilizar en dependencia de si la entrada a procesar es de tipo texto o imagen deben consultar la documentación de Snowflake en el siguiente enlace: [Lista de modelos disponibles](https://docs.snowflake.com/en/sql-reference/functions/ai_complete-prompt-object)

### Ejemplo

En este ejemplo compararemos dos imágenes pasándolas como entrada a la función  `AI_COMPLETE` y preguntando si ambas son fotos de perros.

In [ ]:
USE DATABASE curso_ia;
USE SCHEMA seccion_2;

SELECT AI_COMPLETE('claude-3-5-sonnet',
  PROMPT('¿La imagen {0} y la imagen {1} son ambas de perros?',
    TO_FILE('@imagenes', 'perro1.png'), TO_FILE('@imagenes', 'perro2.jpg')));

### Procesamiento en batch de imágenes desde un directorio o tabla

Para procesar en batch varias imágenes, realizando la misma operación en cada una, debemos almacenar los archivos de imagen en el mismo stage. Luego debemos aplicar la función `AI_COMPLETE` a cada fila de la tabla.

Primero, debemos crear la tabla recuperando las ubicaciones de las imágenes del directorio, convirtiéndolas en objetos `FILE` y almacenando los objetos `FILE` resultantes en una columna de una tabla como se muestra a continuación.

In [ ]:
USE DATABASE curso_ia;
USE SCHEMA seccion_2;

CREATE TABLE image_table AS
    (SELECT TO_FILE('@imagenes', RELATIVE_PATH) AS img FROM DIRECTORY(@imagenes));

Verificamos el contenido de la tabla

In [ ]:
SELECT * FROM curso_ia.seccion_2.image_table;

Luego, debemos aplicar la función `AI_COMPLETE` a la columna que contiene los objetos `FILE`  como se muestra a continuación.

In [ ]:
USE DATABASE curso_ia;
USE SCHEMA seccion_2;

SELECT AI_COMPLETE('claude-3-5-sonnet',
    PROMPT('Clasifique la imagen de entrada {0} en no más de dos palabras. Responde en un JSON.', img)) AS image_classification
FROM image_table;